In [19]:
PRODUCTS_COLLECTION_NAME = "khortoom_products_by_id"
PRODUCTS_TITLE_COLLECTION_NAME = "khortoom_products"
COMMENTS_BODY_COLLECTION_NAME = "khortoom_comments"

PRODUCTS_TITLE_COLLECTION_COSINE_NAME = "khortoom_products_cosine"
COMMENTS_BODY_COLLECTION_COSINE_NAME = "khortoom_comments_cosine"

PRODUCTS_PATH = "../../data/sample/products.csv"
COMMENTS_PATH = "../../data/sample/comments.csv"

In [4]:
import pandas as pd

products = pd.read_csv(PRODUCTS_PATH)
comments = pd.read_csv(COMMENTS_PATH)

In [5]:
from hazm import *

normalizer = Normalizer()

In [6]:
products.drop_duplicates(subset=['id'])
comments.drop_duplicates(subset=['id'])

products["normalized_title_fa"] = products["title_fa"].apply(
    lambda x: str(x)
).apply(
    lambda x: normalizer.normalize(x)
)
products["Category1"] = (
    products["Category1"]
    .apply(lambda x: str(x))
    .apply(lambda x: normalizer.normalize(x))
)
products["Category2"] = (
    products["Category2"]
    .apply(lambda x: str(x))
    .apply(lambda x: normalizer.normalize(x))
)
comments["normalized_body"] = comments["body"].apply(
    lambda x: str(x)
).apply(
    lambda x: normalizer.normalize(x)
)

In [20]:
import os
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings

load_dotenv(".env")

CHROMA_SERVER_AUTH_CREDENTIALS = os.getenv("CHROMA_SERVER_AUTH_CREDENTIALS")

client = chromadb.HttpClient(
    host="https://chroma.liara.run",
    settings=Settings(
        chroma_client_auth_provider="chromadb.auth.token_authn.TokenAuthClientProvider",
        chroma_client_auth_credentials=CHROMA_SERVER_AUTH_CREDENTIALS,
    ),
)

In [21]:
products_by_id_collection =  client.get_or_create_collection(name=PRODUCTS_COLLECTION_NAME)
products_title_collection =  client.get_or_create_collection(name=PRODUCTS_TITLE_COLLECTION_NAME)
comments_body_collection =  client.get_or_create_collection(name=COMMENTS_BODY_COLLECTION_NAME)

In [22]:
products_title_collection_cosine = client.get_or_create_collection(
    name=PRODUCTS_TITLE_COLLECTION_COSINE_NAME,
    metadata={"hnsw:space": "cosine"},
)
comments_body_collection_cosine = client.get_or_create_collection(
    name=COMMENTS_BODY_COLLECTION_COSINE_NAME,
    metadata={"hnsw:space": "cosine"},
)

In [23]:
products = products_title_collection.get(
    limit=1000, include=["embeddings", "metadatas", "documents"]
)

In [24]:
print(len(products["ids"]))
products_title_collection_cosine.upsert(
    ids=products["ids"],
    embeddings=products["embeddings"],
    metadatas=products["metadatas"],
    documents=products["documents"],
)
products_title_collection_cosine.peek(5)

1000


{'ids': ['10000960', '10004289', '10057972', '10058686', '10075881'],
 'embeddings': [[0.08577572554349899,
   0.013564995490014553,
   0.014060872606933117,
   0.034557126462459564,
   -0.00969164352864027,
   -0.01219857856631279,
   0.0197138711810112,
   0.011245392262935638,
   0.03543868660926819,
   -0.030237486585974693,
   -0.01579093188047409,
   0.0013113196473568678,
   -0.021399853751063347,
   -0.02221529744565487,
   0.06783599406480789,
   0.006771478336304426,
   -0.011581487022340298,
   -0.05690465867519379,
   -0.027394458651542664,
   0.012661396525800228,
   0.08374813944101334,
   0.022336510941386223,
   0.03872249647974968,
   -0.018369494006037712,
   0.0573895163834095,
   0.04242504760622978,
   -0.014920393005013466,
   -0.05694873631000519,
   0.032463423907756805,
   0.022005926817655563,
   -0.01294790394604206,
   -0.05478891730308533,
   0.025697456672787666,
   -0.06395713239908218,
   -0.0010675133671611547,
   0.04235892742872238,
   -0.027835238724

In [25]:
comments = comments_body_collection.get(
    limit=1000, include=["embeddings", "metadatas", "documents"]
)

In [26]:
print(len(comments["ids"]))
comments_body_collection_cosine.add(
    ids=comments["ids"],
    embeddings=comments["embeddings"],
    metadatas=comments["metadatas"],
    documents=comments["documents"],
)
comments_body_collection_cosine.peek(5)

342


{'ids': ['10000960', '10075881', '10107051', '10120333', '10218079'],
 'embeddings': [[0.0425678975880146,
   -0.0023980638943612576,
   -0.01772628165781498,
   0.0320190005004406,
   -0.009773241356015205,
   -0.012669016607105732,
   -0.03499751165509224,
   0.043891679495573044,
   0.025462137535214424,
   -0.00807197391986847,
   0.014334087260067463,
   0.00859424751251936,
   -0.0021989792585372925,
   -0.04902133718132973,
   0.07702761888504028,
   0.03301183879375458,
   -0.02033247984945774,
   -0.022442258894443512,
   -0.0029268013313412666,
   -0.05295132100582123,
   0.05514383316040039,
   -0.04192668944597244,
   -0.014789137989282608,
   -0.0429815798997879,
   -0.0006454088143073022,
   0.022462941706180573,
   0.003175010671839118,
   -0.03787260502576828,
   0.044470835477113724,
   0.01810893788933754,
   0.014385798014700413,
   -0.05489562451839447,
   0.049517758190631866,
   -0.03747960552573204,
   0.004191117826849222,
   0.023310991004109383,
   0.004509135

In [20]:
# convert ids to string
product_ids = products["id"].astype(str).to_list()
# flatten the products into a list of dictionaries
metadatas = products.to_dict(orient='records')
documents = products["normalized_title_fa"].to_list()

In [23]:
print("product_ids length: ", len(product_ids))
print("sample product_ids: ", product_ids[:5])
print("metadatas length: ", len(metadatas))
print("sample metadatas: ", metadatas[:5])
print("documents length: ", len(documents))
print("sample documents: ", documents[:5])

product_ids length:  1000
sample product_ids:  ['4807458', '1932905', '11083554', '6129447', '11439540']
metadatas length:  1000
sample metadatas:  [{'id': 4807458, 'title_fa': 'آینه جیبی مدل نوتلا کد 484', 'Rate': 0, 'Rate_cnt': 0, 'Category1': 'برس\u200cها و تجهیزات آرایشی', 'Category2': 'برس ها و تجهیزات آرایشی صورت', 'Brand': 'متفرقه', 'Price': 439000, 'Seller': 'استودیو هنری ژانو', 'Is_Fake': False, 'min_price_last_month': 0, 'sub_category': 'beauty', 'normalized_title_fa': 'آینه جیبی مدل نوتلا کد ۴۸۴'}, {'id': 1932905, 'title_fa': 'اسپری خوشبو کننده بدن زنانه آنیکا مدل La vie est belle حجم 200 میلی لیتر', 'Rate': 84, 'Rate_cnt': 108, 'Category1': 'اسپری', 'Category2': nan, 'Brand': 'آنیکا', 'Price': 850000, 'Seller': 'امپر', 'Is_Fake': False, 'min_price_last_month': 0, 'sub_category': 'beauty', 'normalized_title_fa': 'اسپری خوشبو کننده بدن زنانه آنیکا مدل La vie est belle حجم ۲۰۰ میلی\u200cلیتر'}, {'id': 11083554, 'title_fa': 'دمنوش چای سرگل بهاره و ریشه زعفران و هل حامی - 150 گر

In [24]:
products_by_id_collection.add(ids=product_ids, metadatas=metadatas, documents=documents)

In [26]:
sample_id_result = products_by_id_collection.get(ids=product_ids[:5], include=['metadatas'])
print("sample_id_result: ", sample_id_result)

sample_id_result:  {'ids': ['11083554', '11439540', '1932905', '4807458', '6129447'], 'embeddings': None, 'metadatas': [{'Brand': 'متفرقه', 'Category1': 'مواد غذایی', 'Category2': 'نوشیدنی\u200cهای ارگانیک', 'Is_Fake': False, 'Price': 1355000, 'Rate': 0, 'Rate_cnt': 0, 'Seller': 'ادویه جات حامی', 'id': 11083554, 'min_price_last_month': 0, 'normalized_title_fa': 'دمنوش چای سرگل بهاره و ریشه زعفران و هل حامی - ۱۵۰ گرم', 'sub_category': 'rural goods', 'title_fa': 'دمنوش چای سرگل بهاره و ریشه زعفران و هل حامی - 150 گرم'}, {'Brand': 'لوبلی', 'Category1': 'مواد غذایی', 'Category2': 'خشکبار و آجیل سنتی', 'Is_Fake': False, 'Price': 2850000, 'Rate': 0, 'Rate_cnt': 0, 'Seller': 'بومی و محلی لوبلی', 'id': 11439540, 'min_price_last_month': 0, 'normalized_title_fa': 'بادام\u200cزمینی با پوست نمکی لوبلی - ۷۰۰ گرم', 'sub_category': 'rural goods', 'title_fa': 'بادام زمینی با پوست نمکی لوبلی - 700\xa0گرم'}, {'Brand': 'آنیکا', 'Category1': 'اسپری', 'Is_Fake': False, 'Price': 850000, 'Rate': 84, 'Rate_cn

In [36]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")


openai_client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

def generate_embedding(text):
    embedding = (
        openai_client.embeddings.create(
            input=text, model="text-embedding-3-small"
        )
        .data[0]
        .embedding
    )

    return embedding


def generate_summery(comments):
    # if comments is empty return empty string
    if not comments:
        return ""

    # if comments length is less than 10 append them into a single string and return it
    if len(comments) < 10:
        return " ".join(comments)

    system_prompt = "You are a helpful assistant. You should not engage in a conversation with the user. Your response should be in persian language."

    prompt = f"""I have a list of comments that need to be summarized. Each comment contains various points and details that are crucial for understanding the overall feedback. The summary should be detailed enough to enable effective semantic search for the most relevant results later on.
                You should focus on special points in each comments not general overview like خوب and عالی

                1. The main idea or topic of the comments.
                2. Specific points and details mentioned.
                3. Any notable examples or anecdotes provided.

                Adhere to these guidelines:
                1. Craft a summary that is detailed, thorough, in-depth while maintaining clarity and conciseness.
                2. Incorporate main ideas and essential information, eliminating extraneous language and focusing on critical aspects.
                3. Rely strictly on the provided text, without including external information.
                4. Your response should be in persian language. (زبان فارسی)
                5. Your response should be in a single paragraph and contains only the summary of the comments.


                COMMENTS: 
                {comments}"""

    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )

    return response.choices[0].message.content

In [15]:
# about 10 min for 1000 products

documents = []
ids = []
embeddings = []

for i, row in products.iterrows():
    # if category1 is in row use it otherwise put and empty string
    cat1 = row["Category1"] if row["Category1"] else ""
    cat2 = row["Category2"] if row["Category2"] else ""
    if cat1 == "nan":
        cat1 = ""
    if cat2 == "nan":
        cat2 = ""

    aggregated_title_with_category = (
        row["normalized_title_fa"] + " " + cat1 + " " + cat2
    )
    print(i, aggregated_title_with_category)
    embedding = generate_embedding(aggregated_title_with_category)
    ids.append(str(row["id"]))

    documents.append(aggregated_title_with_category)
    embeddings.append(embedding)

0 آینه جیبی مدل نوتلا کد ۴۸۴ برس‌ها و تجهیزات آرایشی برس‌ها و تجهیزات آرایشی صورت
1 اسپری خوشبو کننده بدن زنانه آنیکا مدل La vie est belle حجم ۲۰۰ میلی‌لیتر اسپری 
2 دمنوش چای سرگل بهاره و ریشه زعفران و هل حامی - ۱۵۰ گرم مواد غذایی نوشیدنی‌های ارگانیک
3 هودی زنانه مدل W ۱۴۶ لباس زنانه هودی زنانه
4 بادام‌زمینی با پوست نمکی لوبلی - ۷۰۰ گرم مواد غذایی خشکبار و آجیل سنتی
5 تفنگ بازی مدل m ۳۵ اسباب‌بازی تفنگ و مبارزه
6 پولیش سازهای زهی لا بلا مدل L ۴ A Premier لوازم جانبی آلات موسیقی سایر لوازم جانبی آلات موسیقی
7 دستبند طلا ۱۸ عیار زنانه فرشته مدل انار بی تو دنیا همه هیچ WBLAGB- ۰۰۰۰۹۰ زیورآلات طلا زنانه دستبند طلا زنانه
8 اکشن فیگور مدل هاسبرو کد ۳۶۳۲ A اسباب‌بازی جغجغه، عروسک و مدل
9 کتاب خودت باش دختر اثر ریچل هالیس کتاب فلسفه و روانشناسی 
10 کتاب جامعه شناسی روسپی‌گری اثر سعید مدنی قهفرخی کتاب علوم اجتماعی 
11 بادی آستین بلند نوزادی مدل ستاره نوزاد لباس نوزاد
12 ست رویه مایو زنانه ناتوسا مدل NTS ۷۴۳ لباس زنانه لباس ورزشی زنانه
13 گلدان میناکاری مسی طرح کمر باریک کد ۱۴۷ محصولات فلزی مین

In [17]:
products_title_collection.add(ids=ids, documents=documents, embeddings=embeddings)

In [20]:
product_comments_map = {}

for _, row in comments.iterrows():
    product_id = str(row["product_id"])
    comment_body = row["normalized_body"]
    if product_id not in product_comments_map:
        product_comments_map[product_id] = []

    product_comments_map[product_id].append(comment_body)

In [21]:
print(product_comments_map)

{'6505713': ['پیشنهاد می\u200cکنم', 'خوب بود من راضی\u200cام', 'مثل همیشه عالی من سال هاس از این مرطوب\u200cکننده استفاده می\u200cکنم', 'در کل خوب بود', 'بسیار خوش\u200cبو', 'بار چندمه که برای دستم سفارش میدم و راضیم', 'قیمت مناسب کیفیت هم بد نیست', 'یه کرم معمولی بود', 'خوبه محصولات هیدرودرم', 'کیفیت خوبی داره فقط یه مقداری از ظرف خالی بود', 'عالی❤️', 'کرم با کیفیتی هست نسبت به بقیه کرم\u200cهای ایرانی', 'من از محصولات هیدرودرم خیلی راضیم. بوی فوق\u200cالعاده داره دستو هم خیلی نرم میکنه و تیره هم نکرد.', 'خوب', 'عالی هستش عالی نرم میکنه', 'خ خوبه ممنونم', 'بار دومی هست ک می\u200cخرم عالیه هم بوش هم نرم کنندگیش', 'مثل همیشه عالی. مرطوب کنندگی و اب رسانی و نرم کنندگی رو واقعا داره', 'بد نیست', 'کرم خیلی خوبیه مرطوب\u200cکننده عالی. یکیشو گرفتم. برا خواهرم هم سفارش دادم. کلا محصولات هیدرودرم. عالیه', 'در کل کرم خوبیه', 'هیدرودرم کرمهای عالی داره', 'برای پوست من مناسب نبود؛ پوستم معمولیه و این کرم خیلی خشک می\u200cکرد', 'خوبه و از خریدش راضی هستم', 'کیفیت نسبت به قیمت خوب', 'کرم مورد علاق

In [39]:
# 10 min runtime for 342 comments (random) of 1000 products
# 6 min runtime for gpt4-o
import tiktoken

product_comments_summary_map = {}

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

for item in product_comments_map.items():
    print(
        f"Remaining products: {len(product_comments_map) - len(product_comments_summary_map)}"
    )
    product_id = item[0]
    product_comments = item[1]

    encoded = encoding.encode(str(product_comments))

    chunk_size = 1
    if len(encoded) > 15000:
        chunk_size = 2
        while True:
            # compute the encoding for each chunk
            for i in range(0, chunk_size):
                chunk = product_comments[
                    i
                    * len(product_comments)
                    // chunk_size : (i + 1)
                    * len(product_comments)
                    // chunk_size
                ]
                encoded = encoding.encode(str(chunk))
                if len(encoded) > 15000:
                    chunk_size *= 2
                    continue
            break

    # for each chunk of comments, get the summary
    for i in range(0, chunk_size):
        chunk = product_comments[
            i
            * len(product_comments)
            // chunk_size : (i + 1)
            * len(product_comments)
            // chunk_size
        ]

        summary = generate_summery(chunk)
        
        if product_id not in product_comments_summary_map:
            product_comments_summary_map[product_id] = []
        product_comments_summary_map[product_id].append(summary)

Remaining products: 342
Remaining products: 341
Remaining products: 340
Remaining products: 339
Remaining products: 338
Remaining products: 337
Remaining products: 336
Remaining products: 335
Remaining products: 334
Remaining products: 333
Remaining products: 332
Remaining products: 331
Remaining products: 330
Remaining products: 329
Remaining products: 328
Remaining products: 327
Remaining products: 326
Remaining products: 325
Remaining products: 324
Remaining products: 323
Remaining products: 322
Remaining products: 321
Remaining products: 320
Remaining products: 319
Remaining products: 318
Remaining products: 317
Remaining products: 316
Remaining products: 315
Remaining products: 314
Remaining products: 313
Remaining products: 312
Remaining products: 311
Remaining products: 310
Remaining products: 309
Remaining products: 308
Remaining products: 307
Remaining products: 306
Remaining products: 305
Remaining products: 304
Remaining products: 303
Remaining products: 302
Remaining produc

In [40]:
import pprint
pprint.pprint(product_comments_summary_map)

{'10000960': ['کارت\u200cپستال زیباییست'],
 '10075881': ['خیلی خوبه. \n'
              'کسایی که آرایش می\u200cکنند میتونند از این محصول به عنوان '
              'پرایمر استفاده کنند. خیلی جوابه عالیه هم بو و هم جنسش بعید '
              'می\u200cدانم اورجینال باشد خوب و عالی خوب'],
 '10107051': ['عالی بود هم\u200cجنس هم سایز جنس معمولی. سایز بزرگتر از اطلاعات '
              'داده\u200cشده هست.'],
 '10120333': ['درباره تاثیرش زوده نظر بدم ولی خیلی چربه و اینکه راحت شسته میشه '
              'و مقاومتی در مقابل تعریق نداره\n'
              'نمیشه به عنوان ضد آفتاب بهش اعتماد کرد حرف نداره خوبه '
              'روشن\u200cکننده تاثیر جزئی داره و لک\u200cهای جدید جای جوش رو '
              'خیلی سریع از بین میبره ولی رو لک\u200cهای قدیمی تاثیر زیادی '
              'نداره یا طول میکشه من یک هفته استفاده کردم و به نظر میاد موثر '
              'باشه\n'
              'سبکه و چرب نیست ولی یک کمی صورت رو براق میکنه به طور واقعی رنگ '
              'پوست را روشن میکنه خوب جذب پوست میشه قیمتش ه

In [42]:
# 3 min runtime for 342 comments of 1000 products (random)
documents = []
ids = []
embeddings = []
metadatas = []

for row in product_comments_summary_map.items():
    print(f"Remaining products: {len(product_comments_summary_map) - len(ids)}")
    # product id
    id = row[0]
    comments = row[1]
    aggregated_comments = " ".join(comments)
    embedding = generate_embedding(aggregated_comments)
    
    metadatas.append({"product_id": id})
    ids.append(id)
    documents.append(aggregated_comments)
    embeddings.append(embedding)

Remaining products: 342
Remaining products: 341
Remaining products: 340
Remaining products: 339
Remaining products: 338
Remaining products: 337
Remaining products: 336
Remaining products: 335
Remaining products: 334
Remaining products: 333
Remaining products: 332
Remaining products: 331
Remaining products: 330
Remaining products: 329
Remaining products: 328
Remaining products: 327
Remaining products: 326
Remaining products: 325
Remaining products: 324
Remaining products: 323
Remaining products: 322
Remaining products: 321
Remaining products: 320
Remaining products: 319
Remaining products: 318
Remaining products: 317
Remaining products: 316
Remaining products: 315
Remaining products: 314
Remaining products: 313
Remaining products: 312
Remaining products: 311
Remaining products: 310
Remaining products: 309
Remaining products: 308
Remaining products: 307
Remaining products: 306
Remaining products: 305
Remaining products: 304
Remaining products: 303
Remaining products: 302
Remaining produc

In [43]:
comments_body_collection.upsert(ids=ids, documents=documents, embeddings=embeddings, metadatas=metadatas)

In [35]:
comments_body_collection.get(ids=ids[:10], include=['documents', 'metadatas'])

{'ids': ['1096464',
  '11398908',
  '1743432',
  '1810442',
  '3358296',
  '350441',
  '5719602',
  '6505713',
  '7409126',
  '7712244'],
 'embeddings': None,
 'metadatas': [{'product_id': '1096464'},
  {'product_id': '11398908'},
  {'product_id': '1743432'},
  {'product_id': '1810442'},
  {'product_id': '3358296'},
  {'product_id': '350441'},
  {'product_id': '5719602'},
  {'product_id': '6505713'},
  {'product_id': '7409126'},
  {'product_id': '7712244'}],
 'documents': ['خلاصه\u200cای از نظرات: این کرم مرطوب\u200cکننده، با قیمت مناسب و کیفیت قابل قبول، از مواد نرم\u200cکننده موثر است اما حاوی پارابن است. بوی آن نسبت به نسخه\u200cهای قبلی تغییر کرده و نرم\u200cکنندگی آن بیشتر از مرطوب\u200cکنندگی است. قدرت جذب خوبی دارد و چربی زیادی ندارد. تاریخ انقضای برخی محصولات ممکن است مورد توجه باشد. بی\u200cصداقتی در ارسال یا اشتباه در بسته بندی نیز ممکن است رخ دهد. از آنجایی که برای پوست خشک موثر بوده و بوی خوبی دارد، مورد پسند برخی مصرف\u200cکنندگان است.',
  'مغازه جادویی خیلی خوبه قلم قویی 